In [ ]:
import os
import cv2
import random
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.models import save_model
from tensorflow.keras import regularizers
from keras.losses import binary_crossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Very Important Step to load the Data correctly

# Follow the README instructions to Load the Data

In [ ]:
#------ PART ONE ------
#Uncomment and use this code if the contents are images in the drive in a floder *which is NOT zipped*
# Change the path before runinng
'''
os.chdir("/content/drive/My Drive/add_your_input_path_here")
destination_folder = "/content/add_your_output_path_here"
os.makedirs(destination_folder, exist_ok=True)
!cp -r . "/content/add_your_output_path_here/"
'''

#------ PART TWO ------
#Uncomment and use this code if the contents are images in the drive in a floder *which is zipped*
# Change the path before runinng
'''
!unzip '/content/drive/MyDrive/add_input_zip_location.zip'
'''

# Change the input path to where the images have been loaded

In [ ]:
dataset = []
labels = []
genuine = 0
fake = 0
#Change you location where you copied the contents or unzipped them
image_path = 'add_your_path_here'
for ic in os.listdir(image_path):
    file_path = os.path.join(image_path, ic)
    circuit = Image.open(file_path)
    resized_circuit = circuit.resize((640,480), Image.LANCZOS)
    circuit_placeholder = Image.new("RGB", (640,480))
    circuit_placeholder.paste(resized_circuit, ((640 - resized_circuit.size[0]) // 2, (480 - resized_circuit.size[1]) // 2))
    circuit_data = np.array(circuit_placeholder)
    circuit_data = circuit_data / 255.0
    if ic[0] == 'A':
        target = 1
        genuine += 1
    elif ic[0] == 'C':
        target = 0
        fake += 1
    dataset.append(circuit_data)
    labels.append(target)
X_test = np.stack(dataset)
y_test = np.stack(labels)

# Do not forget to change path from where your model has to be loaded

In [ ]:
#Provide the path where the model is loaded
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/path_to_your_saved_model')
predicted_labels = loaded_model.predict(X_test)
threshold = 0.5
binary_predictions = np.where(predicted_labels > threshold, 1, 0)
predicted_labels = np.ravel(binary_predictions)
vgg16_test_loss, vgg16_test_accuracy = loaded_model.evaluate(X_test, y_test)
print("VGG16 Test Accuracy:", vgg16_test_accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_test, predicted_labels)


plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['0', '1'], yticklabels=['0', '1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, predicted_labels, labels=[0, 1]).ravel()
print("Positive - Authentic, Negative - Counterfiet")
print("True Negative:", tn)
print("False Positive:", fp)
print("False Negative:", fn)
print("True Positive:", tp)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted_labels))